# Image proc demo
線画生成のデモです。最初に画像URL入力フォームのセルを実行してください。正常に読み込まれれば指定した画像が表示されます。デフォルトの画像は先人に倣ってユーフォニアムの画像です。同じ画像ではありませんが。

参考文献<br>
https://qiita.com/khsk/items/6cf4bae0166e4b12b942<br>
https://qiita.com/koshian2/items/f53088d1dedaa72f9dca


In [0]:
from PIL import Image
from matplotlib import  pyplot as plt
import requests
import io

#@markdown 画像URL入力フォーム
img_src = "https://safebooru.org//samples/1836/sample_1e5bf82da7efac1596270ac57d53fd49ec1c2627.jpg" #@param {type:"string"}

img = Image.open(io.BytesIO(requests.get(img_src).content))
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(img)

In [0]:
#@title Dilation demo
#@markdown dilation paramater
dilation_filtersize = 3 #@param ["3", "5", "7", "9", "11"]  {type:"raw"}
dilation_times = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}

import numpy as np
import tensorflow as tf

def dilation(inputs, filtersize, name='dilation'):
    assert filtersize % 2 == 1, 'filtersize must be odd.'
    with tf.name_scope(name):
        x = tf.nn.max_pool(inputs, [1, filtersize, filtersize, 1], [1, 1, 1, 1], 'SAME')
    return x
holder = tf.placeholder(tf.float32, [1, None, None, 3])
gray_img = tf.image.rgb_to_grayscale(holder)
image = gray_img
for _ in range(dilation_times):
  image = dilation(image, dilation_filtersize)
difference = image - gray_img
image = tf.squeeze(tf.cast(image*255.0, tf.uint8))
difference = tf.squeeze(tf.cast((1.0 - difference)*255.0, tf.uint8))
with tf.Session() as sess:
  input_img = np.array(img.copy()).reshape((1, img.size[1], img.size[0], 3))/255.0
  results = sess.run([image, difference], feed_dict={holder : input_img})
print('dilated image')
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(Image.fromarray(results[0]), cmap='gray')
plt.show()
print('difference image')
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(Image.fromarray(results[1]), cmap='gray')
plt.show()
tf.reset_default_graph()

In [0]:
#@title Erosion demo
#@markdown erosion paramater
erosion_filtersize = 3 #@param ["3", "5", "7", "9", "11"]  {type:"raw"}
erosion_times = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}

import numpy as np
import tensorflow as tf

def erosion(inputs, filtersize, name='erosion'):
    assert filtersize % 2 == 1, 'filtersize must be odd.'
    with tf.name_scope(name):
        x = 1.0 - inputs
        x = tf.nn.max_pool(x, [1, filtersize, filtersize, 1], [1, 1, 1, 1], 'SAME')
        x = 1.0 - x
    return x
holder = tf.placeholder(tf.float32, [1, None, None, 3])
gray_img = tf.image.rgb_to_grayscale(holder)
image = gray_img
for _ in range(dilation_times):
  image = erosion(image, dilation_filtersize)
difference = gray_img - image
image = tf.squeeze(tf.cast(image*255.0, tf.uint8))
difference = tf.squeeze(tf.cast((1.0 - difference)*255.0, tf.uint8))
with tf.Session() as sess:
  input_img = np.array(img.copy()).reshape((1, img.size[1], img.size[0], 3))/255.0
  results = sess.run([image, difference], feed_dict={holder : input_img})
print('eroded image')
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(Image.fromarray(results[0]), cmap='gray')
plt.show()
print('difference image')
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(Image.fromarray(results[1]), cmap='gray')
plt.show()
tf.reset_default_graph()

In [0]:
#@title Adaptive threshold demo
#@markdown threshold paramater
type_ = "gaussian" #@param ["gaussian", "mean"]
filter_size = 3 #@param ["3", "5", "7", "9", "11"] {type:"raw"}
sigma = 1.3 #@param {type:"slider", min:0.01, max:10, step:0.01}
c_ = 2.0 #@param {type:"slider", min:0, max:255, step:0.01}


def gaussian_kernel(ksize=3, sigma=1.3):
    assert ksize % 2 == 1, 'kernel size must be odd.'
    def gausian2d(x, y, sigma):
        z = np.exp(-(x**2 + y**2) / (2 * sigma**2)) / (2 * np.pi * sigma**2)
        return z
    x = y = np.linspace(-sigma, sigma, num=ksize, dtype=np.float32)
    x, y = np.meshgrid(x, y)
    z = gausian2d(x, y, sigma)
    kernel = z / np.sum(z)
    return kernel

def Adaptivethreshold(inputs, filtersize=3, threshold_type='gaussian', sigma=1.3, c=2.0):
    with tf.name_scope('adaptive_threshold'):
        if threshold_type == 'gaussian':
            kernel = tf.constant(gaussian_kernel(ksize=filtersize, sigma=sigma).reshape(filtersize, filtersize, 1, 1), dtype=tf.float32, name='kernel')
        else:
            kernel = tf.ones([filtersize, filtersize, 1, 1]) / (filtersize**2)
        mean = tf.nn.conv2d(inputs, kernel, [1, 1, 1, 1], 'SAME')
        threshold = mean - (c / 255.0)
        image = inputs - threshold
        image = tf.clip_by_value(image, 0.0, 1.0)
        image = tf.ceil(image)
    return image

holder = tf.placeholder(tf.float32, [1, None, None, 3])
gray_img = tf.image.rgb_to_grayscale(holder)
result = Adaptivethreshold(gray_img, filter_size, type_, sigma, c_)
result = tf.cast(tf.squeeze(result*255.0), tf.uint8)

with tf.Session() as sess:
  input_img = np.array(img.copy()).reshape((1, img.size[1], img.size[0], 3))/255.0
  result = sess.run(result, feed_dict={holder : input_img})
print('Result')
plt.figure(figsize=(10,10),dpi=150)
plt.imshow(Image.fromarray(result), cmap='gray')
plt.show()